In [1]:
import attn 
import torch
import numpy as np
torch.set_printoptions(threshold=np.inf)

In [2]:
LAM = attn.LocalAttention(
    neigh_size = 50,
    splits = 4,
    device = 'cpu',
    output_attention=True,
).eval()



In [3]:
attention = attn.AttentionLayer(
    attention = LAM,
    window_size=200,
    d_model = 64,
    n_heads = 4,
    d_k = 64,
    d_v = 64,
    mix = False
).eval()



In [4]:
Q = torch.randn(5, 400, 64).float()   # (batch_size, seq_length, d_model)
K = torch.randn(5, 400, 64).float()   # (batch_size, seq_length, d_model)
V = torch.randn(5, 400, 64).float()   # (batch_size, seq_length, d_model)

output, A = attention(Q, K, V)
print(output[0].shape)

torch.Size([400, 64])


In [5]:
Q_split, K_split, V_split, S, S_masked, A, V_split, output_refined = attention(Q, K, V, debugging=True)

ValueError: too many values to unpack (expected 2)

In [ ]:
Q = torch.randn(5, 200, 64).float()   # (batch_size, seq_length, d_model)
K = torch.randn(5, 200, 64).float()   # (batch_size, seq_length, d_model)
V = torch.randn(5, 200, 64).float()   # (batch_size, seq_length, d_model)

output = attention(Q, K, V)
print(output[0].shape)

In [ ]:
Q = torch.randn(5, 600, 64).float()   # (batch_size, seq_length, d_model)
K = torch.randn(5, 600, 64).float()   # (batch_size, seq_length, d_model)
V = torch.randn(5, 600, 64).float()   # (batch_size, seq_length, d_model)

output = attention(Q, K, V)
print(output[0].shape)

In [ ]:
import sys
torch.full((5, 5), -sys.maxsize)

In [ ]:
window_size = 10
neigh_size = 3
full_mask = torch.tril(torch.ones((window_size,window_size), dtype=torch.double)) - torch.tril(torch.ones((window_size,window_size), dtype=torch.double),diagonal=-neigh_size)


In [ ]:
full_mask

In [ ]:
torch.tril(torch.ones((window_size,window_size), dtype=torch.double))

In [ ]:
torch.tril(torch.ones((window_size,window_size), dtype=torch.double),diagonal=-neigh_size)

In [ ]:
matrix = torch.rand((4,4))

In [ ]:
print(matrix)

In [ ]:
torch.einsum('ii, jj -> ij', matrix, matrix)

In [ ]:
torch.einsum('ii', matrix)

In [ ]:
"""
B S L H E @ B S T H E -> B S L H T
            B S E H T

"""

In [ ]:
torch.arange(6).shape

In [20]:
split_indexes = []
B = 8
L = 100
H = 4
E = 3
neigh_size = 5
l1 = neigh_size
splits = L // neigh_size

for j in torch.arange(0, splits):
    for i in torch.arange(0, l1):
        split_indexes.append(i+l1*j)
split_indexes_Q = torch.tensor(split_indexes).reshape(splits, l1)

In [23]:
# Matrix SI such that for split i, the block i of K has the rows with index in SI[i,:]
split_indexes = [] 
split_indexes_KV_end = None
for j in torch.arange(0, splits):
    for i in torch.arange(0, l1+neigh_size):
        split_indexes.append(i+l1*j-neigh_size)
split_indexes_KV = torch.tensor(split_indexes).reshape(splits, l1+neigh_size)
split_indexes_KV[0] = torch.cat((torch.zeros(neigh_size), torch.arange(0, l1)), 0)

In [24]:
queries = torch.arange(0, B*L*H*E).reshape(B, L, H, E)
Q_expand = queries.unsqueeze(0).expand(splits, B, L, H, E).permute(1,0,2,3,4)
Q_split = Q_expand[:, torch.arange(splits).unsqueeze(1), split_indexes_Q, :].transpose(0,1)


In [26]:
keys = queries.clone()

# Split keys (here we need more rows per block to capture the attention scores of queries and those elements of keys at positions j l1 - neigh_size to j l1.
K_expand = keys.unsqueeze(0).expand(splits, B, L, H, E).permute(1,0,2,3,4)
K_split = K_expand[:, torch.arange(splits).unsqueeze(1), split_indexes_KV, :].transpose(0,1)
K_split[0,:,:neigh_size,:,:] = 0

In [41]:
Q_split.shape

torch.Size([20, 8, 5, 4, 3])

In [39]:
K_split.shape

torch.Size([20, 8, 10, 4, 3])

In [40]:
A.shape

torch.Size([20, 8, 4, 5, 10])

In [44]:
A[2, 1, 2]

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [31]:
print("Q_split shape:", Q_split.shape)
print("K_split shape:", K_split.shape)
S = torch.einsum("sblhe,sbthe->sbhlt", Q_split, K_split)
print("S shape:", S.shape)

Q_split shape: torch.Size([20, 8, 5, 4, 3])
K_split shape: torch.Size([20, 8, 10, 4, 3])
S shape: torch.Size([20, 8, 4, 5, 10])


In [32]:
A = torch.softmax(S.float(), dim=-1)


In [ ]:
torch.cat(A, 2)

In [ ]:
A.shape